In [78]:
import os

get_ipython().run_line_magic('load_ext', 'autoreload')
get_ipython().run_line_magic('autoreload', '2')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [79]:
from src.chromaDB_setup import *
from src.github_parser import *
from src.embedding import *
import chromadb
from chromadb.config import Settings
import numpy as np

In [80]:
# ("khoj-ai/khoj")

In [81]:
repo_files = get_repo_files("khoj-ai/khoj")

Error processing file documentation/assets/img/dream_house.png: unsupported encoding: none
Error processing file documentation/assets/img/example_search_model_admin_settings.png: unsupported encoding: none
Error processing file documentation/assets/img/khoj-logo-sideways.svg: unsupported encoding: none
Error processing file documentation/assets/img/plants_i_got.png: unsupported encoding: none
Error processing file documentation/assets/img/quadratic_equation_khoj_web.gif: unsupported encoding: none
Error processing file documentation/assets/img/using_khoj_for_studying.gif: unsupported encoding: none
Error processing file src/interface/desktop/assets/icons/khoj-logo-sideways.svg: unsupported encoding: none
Error processing file tests/data/images/nasdaq.jpg: unsupported encoding: none


In [82]:
chunked_docs = chunk_repository_files(repo_files)

textual_embeddings = np.array([embed_textual_metadata(doc["content"]) for doc in chunked_docs['textual_chunks']])

# Process code chunks with CodeBERT
code_embeddings = np.array([generate_code_embedding(doc["content"]) for doc in chunked_docs['code_chunks']])

embedded_chunks = {
    'textual_embeddings': np.array(textual_embeddings),
    'code_embeddings': np.array(code_embeddings)
}

In [ ]:
chrombaDb_collections = setup_chroma_collections(chunked_docs, embedded_chunks)

In [91]:
def retrieve_relevant_chunks(query, collections, top_k=3):
    textual_results = collections['textual_collection'].query(
        query_embeddings=[embed_textual_metadata(query)],
        n_results=top_k * 2,
        include=['documents', 'metadatas', 'distances']
    )

    code_results = collections['code_collection'].query(
        query_embeddings=generate_code_embedding(query).tolist(),
        n_results=top_k * 2,
        include=['documents', 'metadatas', 'distances']

    )

    def process_results(results):

        if "distances" not in results or not results["distances"]:
            print("Warning: 'distances' key missing or empty in results. Returning empty list.")
            return []
        # Calculate scores based on distances
        distances = np.array(results["distances"][0])
        scores = 1 - distances  # Invert distance to get similarity score
        return sorted(zip(scores, results["documents"], results["metadatas"]), reverse=True)[:top_k]

    top_textual = process_results(textual_results)
    top_code = process_results(code_results)

    return {"textual": top_textual, "code": top_code}

query = "What is this repo supposed to do?"
retrieve_relevant_chunks(query, chrombaDb_collections)


{'textual': [(np.float64(0.5679551064968109),
   ["# Github integration :::warning[Unmaintained] The Github integration is not maintained. We are considering deprecating it. It doesn't seem used by many folks and its cumbersome for us to maintain. ::: The Github integration allows you to index as many repositories as you want. It's currently default configured to index all Markdown/Org/Text files in each repository. For large repositories, this takes a fairly long time, but it works well for smaller projects. # Configure your settings 1. Go to [https://app.khoj.dev/settings](https://app.khoj.dev/settings) and enter in settings for the data sources you want to index. You'll have to specify the file paths. ## Use the Github plugin 1. Generate a [classic PAT (personal access token)](https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens) from [Github](https://github.com/settings/tokens) with `repo` and `admin:org` scopes at leas